In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Новая папка\База.xlsx")

In [3]:
df['Дата обновления'] = pd.to_datetime(df['Дата обновления'], errors='coerce')


In [9]:
# df = df[df['Округ'].isin(['НАО', 'ТАО'])]

In [8]:
# Шаг 1 — исключаем квартиры от Фонда Реновации
df_filtered = df[df['Девелопер'] != 'Фонд реновации'].copy()

# Шаг 2 — добавляем числовой месяц
df_filtered['месяц'] = df_filtered['Дата обновления'].dt.month

# Шаг 3 — группировка по проекту и девелоперу + подсчёт квартир по месяцам
project_month_counts = (
    df_filtered
    .groupby(['Название проекта', 'Девелопер', 'месяц'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={9: 'сентябрь', 10: 'октябрь'})
)
project_month_counts = project_month_counts.rename(columns={9: 'сентябрь', 10: 'октябрь'})

# Шаг 4 — фильтрация:
# - минимум 5 квартир в июне
# - июль не ноль
# - изменение >= 80% в любую сторону
filtered = project_month_counts[
    (project_month_counts['сентябрь'] >= 1) &
    (project_month_counts['октябрь'] >= 0) &
    (
        (project_month_counts['октябрь'] >= project_month_counts['сентябрь']) |
        (project_month_counts['октябрь'] <= project_month_counts['сентябрь'])
    )
]

# Шаг 5 — читаемый вывод
result = [
    f"{project} — сентябрь: {row['сентябрь']}, октябрь: {row['октябрь']}"
    for project, row in filtered.iterrows()
]

# Вывод
for line in result:
    print(line)

('1-й Донской', 'ФСК') — сентябрь: 736, октябрь: 681
('1-й Измайловский', 'ФСК') — сентябрь: 347, октябрь: 262
('1-й Ленинградский', 'ФСК') — сентябрь: 123, октябрь: 108
('1-й Лермонтовский', 'ФСК') — сентябрь: 179, октябрь: 115
('1-й Саларьевский', 'ФСК') — сентябрь: 227, октябрь: 230
('1-й Химкинский', 'ФСК') — сентябрь: 312, октябрь: 337
('1-й Шереметьевский', 'ФСК') — сентябрь: 518, октябрь: 380
('1-й Южный', 'ФСК') — сентябрь: 538, октябрь: 477
('1-й Ясеневский', 'ФСК') — сентябрь: 341, октябрь: 282
('1864 Резиденция', 'Сбер Капитал') — сентябрь: 28, октябрь: 24
('2-й Иртышский', 'ПИК') — сентябрь: 282, октябрь: 221
('31 Квартал', 'ПРОФИ-Инвест') — сентябрь: 195, октябрь: 170
('7 небо', 'ПРОФИ-Инвест') — сентябрь: 197, октябрь: 174
('8 Кленов', 'Сити 21 век') — сентябрь: 222, октябрь: 177
('AFI Park Воронцовский', 'AFI') — сентябрь: 26, октябрь: 15
('AHEAD', 'VOS’HOD') — сентябрь: 21, октябрь: 22
('ARTEL', 'Stenoy') — сентябрь: 98, октябрь: 96
('Adres', 'СЗ Т-ОТЕЛЬ') — сентябрь: 1

In [16]:
filtered.head()

,месяц,сентябрь,октябрь,Разница
Название проекта,Девелопер,,,
1-й Донской,ФСК,736,681,55
1-й Измайловский,ФСК,347,262,85
1-й Ленинградский,ФСК,123,108,15
1-й Лермонтовский,ФСК,179,115,64
1-й Саларьевский,ФСК,227,230,-3


In [11]:
filtered['Разница'] = filtered['сентябрь'] - filtered['октябрь']

C:\Users\m.olshanskiy\AppData\Local\Temp\ipykernel_21932\3930492352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Разница'] = filtered['сентябрь'] - filtered['октябрь']


In [14]:
print(filtered['Разница'].sum())

10057


In [17]:
filtered.to_excel(r"C:\Users\m.olshanskiy\Desktop\разница.xlsx")